In [ ]:
#注意力机制的实现
class CoordAtt(nn.Module):
    def __init__(self, inp, oup, reduction=32):   #输入通道数 输出通道数 缩减比例
        super(CoordAtt, self).__init__()
        # self.pool_w = nn.AdaptiveAvgPool1d(1)
        self.pool_w = nn.AdaptiveMaxPool1d(1)    #自适应最大池化 保证可以有不同维度的输入
        mip = max(6, inp // reduction)      #输入通道数衰减 但不小于6
        self.conv1 = nn.Conv1d(inp, mip, kernel_size=1, stride=1, padding=0)
        self.bn1 = nn.BatchNorm1d(mip, track_running_stats=False) #不追踪均值方差的平均值信息，训练时使用当前的，测试时也用当前的，适用于小样本
        self.act = MetaAconC(mip) #自定义了一个激活函数 赋值给self.act
        self.conv_w = nn.Conv1d(mip, oup, kernel_size=1, stride=1, padding=0)
##总结：引入坐标注意力机制，帮助网络更好地捕获输入特征的空间关系

    def forward(self, x):
        identity = x   #保存输入 便于之后的残差连接
        n, c, w = x.size() #获取输入张量的维度信息 n为batch大小
        x_w = self.pool_w(x) #输入进行自适应最大池化
        y = torch.cat([identity, x_w], dim=2) #原始输入和池化后的输入拼接 
        y = self.conv1(y)  
        y = self.bn1(y)
        y = self.act(y) #通过自定义的激活函数
        x_ww, x_c = torch.split(y, [w, 1], dim=2) #y沿宽度维度切分为2部分，包含前w个位置的信息，一个位置的信息两种
        a_w = self.conv_w(x_ww)
        a_w = a_w.sigmoid()
        out = identity * a_w
        return out
